In [1]:
import pandas as pd
import xarray as xr

In [ ]:
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes16/l2/data/xrsf-l2-avg1m_science/sci_xrsf-l2-avg1m_g16_s20170207_e20231002_v2-2-0.nc
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes16/l2/data/xrsf-l2-flsum_science/sci_xrsf-l2-flsum_g16_s20170209_e20231002_v2-2-0.nc
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes17/l2/data/xrsf-l2-avg1m_science/sci_xrsf-l2-avg1m_g17_s20180601_e20230110_v2-2-0.nc
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes17/l2/data/xrsf-l2-flsum_science/sci_xrsf-l2-flsum_g17_s20180601_e20230110_v2-2-0.nc
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes18/l2/data/xrsf-l2-avg1m_science/sci_xrsf-l2-avg1m_g18_s20220902_e20231002_v2-2-0.nc
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes18/l2/data/xrsf-l2-flsum_science/sci_xrsf-l2-flsum_g18_s20220905_e20231002_v2-2-0.nc

In [ ]:
file_avg16 = '/content/sci_xrsf-l2-avg1m_g16_s20170207_e20231002_v2-2-0.nc'
file_sum16 = '/content/sci_xrsf-l2-flsum_g16_s20170209_e20231002_v2-2-0.nc'
file_avg17 = '/content/sci_xrsf-l2-avg1m_g17_s20180601_e20230110_v2-2-0.nc'
file_sum17 = '/content/sci_xrsf-l2-flsum_g17_s20180601_e20230110_v2-2-0.nc'
file_avg18 = '/content/sci_xrsf-l2-avg1m_g18_s20220902_e20231002_v2-2-0.nc'
file_sum18 = '/content/sci_xrsf-l2-flsum_g18_s20220905_e20231002_v2-2-0.nc'

In [ ]:
ds16_avg = xr.open_dataset(file_avg16)
ds16_flare = xr.open_dataset(file_sum16)

ds17_avg = xr.open_dataset(file_avg17)
ds17_flare = xr.open_dataset(file_sum17)

ds18_avg = xr.open_dataset(file_avg18)
ds18_flare = xr.open_dataset(file_sum18)

In [ ]:
def process_telescope_data(file_path, flare_summary_path, telescope_number):
    ds = xr.open_dataset(file_path)
    ds2 = xr.open_dataset(flare_summary_path)

    # Convert xarray DataArrays to pandas DataFrames
    ds = ds[['xrsa_flux', 'xrsb_flux', 'time']].to_dataframe().reset_index()
    ds2 = ds2[['status', 'time']].to_dataframe().reset_index()

    # Add telescope identifier
    ds['telescope'] = telescope_number
    ds2['telescope'] = telescope_number

    return ds, ds2

# Process data for each telescope
ds_16, ds2_16 = process_telescope_data(file_avg16, file_sum16, 16)
ds_17, ds2_17 = process_telescope_data(file_avg17, file_sum17, 17)
ds_18, ds2_18 = process_telescope_data(file_avg18, file_sum18, 18)

In [ ]:
# Concatenate data from all telescopes
ds = pd.concat([ds_16, ds_17, ds_18])
ds2 = pd.concat([ds2_16, ds2_17, ds2_18])

In [ ]:
ds.head()

In [ ]:
ds2.head()

In [ ]:
merged_goes_data = pd.merge(ds, ds2, on=['time', 'telescope'], how='outer')

In [ ]:
merged_goes_data.head()

In [ ]:
merged_goes_data.to_csv('merged_goes_f16t18_data.csv', index=False)